In [1]:
from pyspark.ml import clustering
from pyspark.ml import feature
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql import functions as sf
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.feature import RegexTokenizer
import requests
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import concat, col, lit, lower
from pyspark.sql.functions import isnan, when, count, col, isnull
from pyspark.sql.functions import concat_ws
from  pyspark.sql.functions import abs
# seting master("local[*]") enables multicore processing on all available logical cores on your machine
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [3]:
airlines_df = spark.read.csv(get_training_filename('airlines.csv'), header=True, inferSchema=True)
airports_df = spark.read.csv(get_training_filename('airports.csv'), header=True, inferSchema=True)
flights_df = spark.read.csv(get_training_filename('flights.csv'), header=True, inferSchema=True)

In [4]:
flights_df = flights_df.select('MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'CANCELLED')

flights_df = flights_df.fillna( { 'DEPARTURE_DELAY':0 } )

from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=[ 0, 100, 1000, float('Inf') ],inputCol="DISTANCE", outputCol="Distance_Bucket")
flights_df = bucketizer.setHandleInvalid("keep").transform(flights_df)

from pyspark.sql.functions import udf
from pyspark.sql.types import *

t = {0.0:"Short", 1.0: "Medium", 2.0:"Long"}
udf_foo = udf(lambda x: t[x], StringType())
flights_df = flights_df.withColumn("Flight_Distance", udf_foo("Distance_Bucket"))

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="AIRLINE", outputCol="Airline_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="Airline_Numeric", outputCol="Airline_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="ORIGIN_AIRPORT", outputCol="OA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="OA_Numeric", outputCol="Origin_Airport_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="DA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="DA_Numeric", outputCol="Destination_Airport_OHE")
flights_df= encoder.transform(flights_df)

flights_df = flights_df.drop('ARRIVAL_DELAY')

In [5]:
training_df, testing_df = flights_df.randomSplit([0.9, 0.1], seed=5)

In [6]:
major_df = training_df.filter(col("CANCELLED") == 0)
minor_df = training_df.filter(col("CANCELLED") == 1)
ratio = major_df.count()/minor_df.count()
sampled_majority_df = major_df.sample(False, 1/ratio, seed=5)
combined_df_2 = sampled_majority_df.unionAll(minor_df)

In [7]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

va = VectorAssembler(
    inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "DEPARTURE_DELAY"], outputCol="features")

rf = RandomForestClassifier(labelCol="CANCELLED", featuresCol="features")

rf_pipeline = Pipeline(stages=[va, rf])
rf_model = rf_pipeline.fit(combined_df_2)

prediction = rf_model.transform(testing_df)

In [8]:
bce = BinaryClassificationEvaluator(labelCol='CANCELLED')
score_auc = bce.evaluate(prediction, {bce.metricName: "areaUnderROC"} )
print(score_auc)

0.88263604140408

In [9]:
mce = evaluation.MulticlassClassificationEvaluator(labelCol="CANCELLED", metricName="weightedPrecision")
score_Precision = mce.evaluate(prediction)
print(score_Precision)

0.9843681211795533

In [10]:
mce = evaluation.MulticlassClassificationEvaluator(labelCol="CANCELLED", metricName="weightedRecall")
score_Recall = mce.evaluate(prediction)
print(score_Recall)

0.6457616585292159

In [11]:
mce = evaluation.MulticlassClassificationEvaluator(labelCol="CANCELLED", metricName="f1")
score_f1 = mce.evaluate(prediction)
print(score_f1)

0.770098327974035

In [12]:
mce = evaluation.MulticlassClassificationEvaluator(labelCol="CANCELLED", metricName="accuracy")
score_accuracy = mce.evaluate(prediction)
print(score_accuracy)

0.6457616585292159

In [13]:
scores = [score_auc, score_Recall, score_accuracy, score_f1, score_Precision]
metricName = ['AUC', 'Recall', 'Accuracy', 'f1', 'Precision']

metric_df = pd.DataFrame(zip(metricName,scores),index=[1,2,3,4,5],columns=['Metric Name', 'Score'])
metric_df

,Metric Name,Score
1,AUC,0.882636
2,Recall,0.645762
3,Accuracy,0.645762
4,f1,0.770098
5,Precision,0.984368
